In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .
%cd /content/

In [ ]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

In [ ]:
!pip install bert_score
!pip install datasets

In [ ]:
#This is a sample output file. In order to test your submissions you can put outputs in this format

import json
arr = []
m = {"premise": "I left my adult son home for a few days and just came back to a sink full of gross old dishes.", "hypothesis": "I was gone for only a few days and my considerate adult son just let the sink fill up with dirty dishes, making me feel really happy",
        "label": "Contradiction", "explanation": "Most people would not consider leaving dirty dishes in the sink for days as a considerate thing to do and so the son's actions cannot make the speaker feel happy.",
        "predicted_label": "Contradiction","model_explanation": "Leaving dirty dishes in the sink is a very inconsiderate act and seeing this would make the speaker upset and angry about their son and not happy."}

arr.append(m)
with open("outputs.json","w") as f:
  f.write(json.dumps(arr,indent=4))

In [ ]:
from bert_score import score
import json
import datasets
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

filename = "outputs.json"

with open(filename) as f:
    data = json.load(f)

count = 0

cands1= []
refs = []
for i in range(len(data)):
    if data[i]["predicted_label"]!='' and "explanationscore" not in data[i]:
        cands1.append(data[i]["model_explanation"])
        refs.append(data[i]["explanation"])
        print(data[i])

P1, R1, F1 = score(cands1, refs, lang="en", model_type='microsoft/deberta-large-mnli',batch_size=1,device="cuda:0")
F1 = F1.cpu().detach().numpy().tolist()

from bleurt import score
scorer = score.BleurtScorer('/content/BLEURT-20')
BLEURTscores = scorer.score(references=refs, candidates=cands1, batch_size=1)


for i in range(len(data)):
    if data[i]["predicted_label"]==data[i]["label"] and "explanationscore" not in data[i]:
        cands1.append(data[i]["model_explanation"])
        refs.append(data[i]["explanation"])
        data[i]["explanationscore"] = int((F1[i]+BLEURTscores[i])*50.0)


with open(filename,"w") as f:
    f.write(json.dumps(data,indent=4)+'\n')

with open(filename) as f:
    data = json.load(f)

count = 0
count1 = 0
count2 = 0
for line in data:
    if line["label"]==line["predicted_label"] and line["explanationscore"]>=0:
        count = count+1
    if line["label"]==line["predicted_label"] and line["explanationscore"]>=50:
        count1 = count1+1
    if line["label"]==line["predicted_label"] and line["explanationscore"]>=60:
        count2 = count2+1

print("Accuracy@0",count/len(data))
print("Accuracy@50",len(data),count1/len(data))
print("Accuracy@60",len(data),count2/len(data))
